In [1]:
import numpy as np
import copy
import random
import operator
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
distances = [
    [0,32,39,42,29,35],
    [32,0,36,27,41,25],
    [39,36,0,28,33,40],
    [42,27,28,0,27,38],
    [29,41,33,27,0,26],
    [35,25,40,38,26,0]
]

cities = [1,2,3,4,5,6]

def distance_calc(me,neighbour): #returns distances from the distance matrix
    return distances[me-1][neighbour-1]

class city:
    
    def __init__(self,me):
        self.me = me-1
    
    def d(self,neighbour):
        return distances[self.me][neighbour-1]
    def num(self):
        return self.me
    
cityList = []
for entry in cities:
    cityList.append(city(entry))
    
class route:
    
    def __init__(self,cities,rand):
        self.route = cities
        if(rand == 1):
            self.route = random.sample(cities,len(cities))
        
    def getRoute(self):
        return self.route
    
    def getCities(self): #returns a list of ints for breeding and display purposes
        lst = []
        for city in self.route:
            if(type(city)==int):
                lst.append(city)
                continue
            lst.append(city.num()+1)
        return lst
    
    
    def calcFitness(self): #calculates the fitness of the route as 1/distance
        routeDistance = 0
        dist = 0
        for i in range(len(self.route)):
            dist = 0
            if(type(self.route[0])==int):
                if(i==len(self.route)-1):
                    dist = distance_calc(self.route[i],self.route[0])
                    routeDistance += dist
                    self.fitness = float(1/float(routeDistance))
                    return self.fitness
                dist = distance_calc(self.route[i],self.route[i+1])
                routeDistance += dist
            
            else:
                if(i==len(self.route)-1):
                    dist = distance_calc(self.route[i].num()+1,self.route[0].num()+1)
                    routeDistance += dist
                    self.fitness = float(1/float(routeDistance))
                    return self.fitness
                dist = distance_calc(self.route[i].num()+1,self.route[i+1].num()+1)
                routeDistance += dist
        return self.fitness

def selectRoutes(routeList): #sorts different routes into order with highest fitness first
    sortedFitness = {}
    for count,entry in enumerate(routeList):
        sortedFitness[count] = entry.calcFitness()
    temp = copy.copy(sortedFitness)
    sortedFitness = sorted(temp.items(),key = operator.itemgetter(1), reverse = True)
    sortedFitness = list(sortedFitness)
    sortedFitness = [list(element) for element in sortedFitness]
    return sortedFitness

def tournament_select(sortedFitness,final_list): #tournament with 3 individuals randomly selected with proportion to their fitness
    #returns all 6 parents
    parent_list = []   
    for i in range(6):
        tournament_list = []
        for k in range(3):
            selection_number = random.random()
            if(selection_number<sortedFitness[0][1]):
                selection_number = random.random()
            if(selection_number==1.0):
                chosen_route = sortedFitness[-1][0]
            for element in sortedFitness:
                if(element[1]<=selection_number):
                    chosen_route = element[0]

            for trip in sortedFitness:
                if(chosen_route==trip[0]):
                    competitor = final_list[chosen_route]
                    tournament_list.append(competitor)

        tournament_list = sorted(tournament_list,key=lambda entry: entry.fitness,reverse=True) 
        winner = tournament_list[0] #sort tournament list and select highest fitness
        parent_list.append(winner)
    print("size of parents = ",len(parent_list))
    return parent_list

def crossover(parent_list): #handles the 2 point crossover process
    #returns all offspring from all parents
    offspring = []
    for i in range(int(len(parent_list)/2)):
        parent1 = parent_list[2*i].getCities()
        parent2 = parent_list[2*i+1].getCities()
        for j in range(2):
            if(parent1 == parent2):
                print("Parent 1:",parent1)
                print("          --------X--------")
                print("Parent 2:",parent2)
                print("Identical parents \n")
                offspring.append(route(parent1,rand=0))
                break
            child = ['x','x','x','x','x','x']
            crossover1 = random.randrange(len(parent1)/2)
            crossover2 = random.randrange(len(parent2))
            while(crossover2<=crossover1): #need different crossover points
                crossover2 = random.randrange(len(parent2))
            print("Parent 1:",parent1)
            print("          --------X--------")
            print("Parent 2:",parent2)
            print("     Crossovers at ",crossover1,"and ",crossover2)
            child[:crossover1] = parent1[:crossover1]
            child[crossover2:] = parent1[crossover2:]
            print("Showing cut points for child:")
            print(child)
            for count,k in enumerate(child):
                if(k=='x'):
                    for c in parent2:
                        if(c not in child):
                            child[count] = c
                            break
            print(child)
            offspring.append(route(child,rand=0))
            print()
    return offspring
    


In [3]:
route_list = []
population_size = 8
for i in range(population_size):
    route_list.append(route(cityList,rand=1))
print("Initial population (8 random):")
for trip in route_list:
    print(trip.getCities())

Initial population (8 random):
[2, 4, 5, 3, 1, 6]
[2, 4, 1, 6, 5, 3]
[3, 2, 5, 6, 1, 4]
[3, 4, 5, 1, 2, 6]
[6, 4, 1, 3, 2, 5]
[3, 4, 1, 5, 6, 2]
[5, 4, 3, 1, 2, 6]
[6, 4, 1, 2, 3, 5]


In [4]:
incumbent_counter = 0
final_list = route_list
bestFitness = 0
iteration = 0
sortedFitness = selectRoutes(final_list)
bestFitness  = sortedFitness[0][1]
parent_list=[]

while(incumbent_counter<9):
    iteration +=1
    print("________________________________")
    print("Iteration Number:",iteration)
    sortedFitness = selectRoutes(final_list)
    newBest = sortedFitness[0][1]
    if(bestFitness>=newBest):
        incumbent_counter+=1
    else:
        bestFitness=newBest
        incumbent_counter=0
    print("Current Population:")
    mean_fitness = 0
    for thing in final_list:
        mean_fitness+=thing.calcFitness()
        print(thing.getCities(),"fitness = ", thing.calcFitness())
    mean_fitness = mean_fitness/len(final_list)
    print()
    print("best fitness:",bestFitness)
    print()
    print("mean_fitness = ",mean_fitness )

    totalFitness = 0
    for element in sortedFitness:
        totalFitness += element[1]

    for element in sortedFitness:
        element[1] = element[1]/totalFitness

    for count,element in enumerate(sortedFitness):

        if(count>=1):
            element[1] += sortedFitness[count-1][1]

    #Result of the above is to have a cumulative total of fitness
    #which allows easy random selection proportional to fitness values

    #Generating parent list through tournament
    print(sortedFitness)
    parent_list = tournament_select(sortedFitness,final_list)
    for parent in parent_list:
        print(parent.getCities())
    print()
    #Starting crossover
    offspring = crossover(parent_list)

    #The below mutation section was not made modular since it changes just one element of the children.
    #If the mutation occured more often it would make sense to make a separate function

    mutation_selector = random.randrange(len(offspring))
    mutated = offspring[mutation_selector].getCities() #select the array of ints to be mutated

    print("Child from all offspring selected for mutation --->",mutated)
    print()
    swapped1 = random.randrange(len(mutated))
    swapped2 = random.randrange(len(mutated))
    while(swapped2==swapped1):
        swapped2 = random.randrange(len(mutated))
    temp = mutated[swapped1]
    mutated[swapped1] = mutated[swapped2]
    mutated[swapped2] = temp
    print("Swapping locations:", swapped1, "and", swapped2)
    print("Result of mutation process of child            --->",mutated)
    print()
    offspring[mutation_selector] = route(mutated,rand=0)

    for parent in parent_list:
        offspring.append(parent)

    sortedOffspring = selectRoutes(offspring) #Order all offspring and parents by fitness

    final_list = []
    for count,element in enumerate(sortedOffspring): #select 8 most fit to go into final_list and continue to next population
        if(count>=population_size):
            break
        final_list.append(offspring[element[0]])
    print("Best Distance so far:",1/bestFitness)
    print("Best Distance this generation:",1/newBest)
    print("________________________________")
    
    

________________________________
Iteration Number: 1
Current Population:
[2, 4, 5, 3, 1, 6] fitness =  0.005376344086021506
[2, 4, 1, 6, 5, 3] fitness =  0.005025125628140704
[3, 2, 5, 6, 1, 4] fitness =  0.004807692307692308
[3, 4, 5, 1, 2, 6] fitness =  0.0055248618784530384
[6, 4, 1, 3, 2, 5] fitness =  0.0045045045045045045
[3, 4, 1, 5, 6, 2] fitness =  0.005376344086021506
[5, 4, 3, 1, 2, 6] fitness =  0.005649717514124294
[6, 4, 1, 2, 3, 5] fitness =  0.004830917874396135

best fitness: 0.005649717514124294

mean_fitness =  0.005136938484919249
[[6, 0.13747773919014294], [3, 0.27191729629873573], [0, 0.4027428868183879], [5, 0.53356847733804], [1, 0.6558476724971118], [7, 0.7734011016596978], [2, 0.890389370105156], [4, 0.9999999999999997]]
size of parents =  6
[3, 4, 1, 5, 6, 2]
[3, 4, 1, 5, 6, 2]
[3, 4, 5, 1, 2, 6]
[5, 4, 3, 1, 2, 6]
[2, 4, 5, 3, 1, 6]
[3, 4, 5, 1, 2, 6]

Parent 1: [3, 4, 1, 5, 6, 2]
          --------X--------
Parent 2: [3, 4, 1, 5, 6, 2]
Identical parents 

P